In [22]:
from scapy.all import *
import pandas as pd # Pandas - Create and Manipulate DataFrames
import binascii # Binary to Ascii 
%matplotlib inline


def pkt_callback(pkt):
    pkt.show() # debug statement

#sniff(iface="s1-eth1", prn=pkt_callback, filter="tcp", store=0,count=5)
pcap = sniff(iface="s1-eth1", prn=pkt_callback, filter="tcp",count=3)

# Collect field names from IP/TCP/UDP (These will be columns in DF)
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Create blank DataFrame
df = pd.DataFrame(columns=dataframe_fields)

for packet in pcap[IP]:
    # Field array for each row of DataFrame
    field_values = []
    # Add all IP fields to dataframe
    for field in ip_fields:
        if field == 'options':
            # Retrieving number of options defined in IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
#                field_values.append(len(packet[layer_type].fields[field]))
                field_values.append(packet[layer_type].fields[field][0])
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Append payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Add row to DF
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)
    
# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

# Return Src Address, Dst Address, Src Port, Dst Port
df[['src','dst','sport','dport','flags','options']]
# df.columns
# df[df.columns]

###[ Ethernet ]### 
  dst       = 2a:9c:04:03:ac:4d
  src       = 76:3e:87:85:6f:66
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 51327
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = tcp
     chksum    = 0x5e3a
     src       = 10.0.0.1
     dst       = 10.0.0.2
     \options   \
###[ TCP ]### 
        sport     = 38616
        dport     = 5005
        seq       = 3526314597
        ack       = 0
        dataofs   = 10
        reserved  = 0
        flags     = S
        window    = 29200
        chksum    = 0x1431
        urgptr    = 0
        options   = [('MSS', 1460), ('SAckOK', ''), ('Timestamp', (156598810, 0)), ('NOP', None), ('WScale', 9)]

###[ Ethernet ]### 
  dst       = 76:3e:87:85:6f:66
  src       = 2a:9c:04:03:ac:4d
  type      = 0x800
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 60
     id        = 0
     flags     

,src,dst,sport,dport,flags,flags,options,options
0,10.0.0.1,10.0.0.2,38616,5005,DF,S,0,"(MSS, 1460)"
1,10.0.0.2,10.0.0.1,5005,38616,DF,SA,0,"(MSS, 1460)"
2,10.0.0.1,10.0.0.2,38616,5005,DF,A,0,"(NOP, None)"
